In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'1.12.0'

### numpy

In [3]:
features = np.arange(0, 10)
labels = np.arange(10, 20)

In [4]:
input_fn = tf.estimator.inputs.numpy_input_fn(
                x=features,
                y=labels,
                batch_size=2,
                num_epochs=2,
                shuffle=False)

In [16]:
features_op, labels_op = input_fn()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    coordinator = tf.train.Coordinator()
    _ = tf.train.start_queue_runners(coord=coordinator)
    for i in range(3):
        print(sess.run((features_op, labels_op)))

({'features': array([0, 1])}, array([10, 11]))
({'features': array([2, 3])}, array([12, 13]))
({'features': array([4, 5])}, array([14, 15]))


### pandas

In [6]:
df = pd.DataFrame(
    {'features': np.arange(0, 10),
     'labels': np.arange(10, 20)}, )

In [7]:
input_fn = tf.estimator.inputs.pandas_input_fn(
                x=df[['features']],
                y=df['labels'],
                batch_size=2,
                num_epochs=2,
                shuffle=False)

In [17]:
features_op, labels_op = input_fn()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    coordinator = tf.train.Coordinator()
    _ = tf.train.start_queue_runners(coord=coordinator)
    for i in range(3):
        print(sess.run((features_op, labels_op)))

({'features': array([0, 1])}, array([10, 11]))
({'features': array([2, 3])}, array([12, 13]))
({'features': array([4, 5])}, array([14, 15]))


### tf.data tensor (combines the input and returns a dataset with a single element)

In [9]:
features = tf.range(0, 10)
labels = tf.range(10, 20)

In [10]:
dataset = tf.data.Dataset.from_tensors((features, labels))
dataset = dataset.repeat(2).batch(2)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [11]:
with tf.Session() as sess:
    print(sess.run(next_element))

(array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]), array([[10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]))


### tf.data tensor_slices

In [12]:
features = np.arange(0, 10)
labels = np.arange(10, 20)

In [13]:
dataset = tf.data.Dataset.from_tensor_slices(({'features': features}, labels))
dataset = dataset.repeat(2).batch(2)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [14]:
with tf.Session() as sess:
    for i in range(3):
        print(sess.run(next_element))

({'features': array([0, 1])}, array([10, 11]))
({'features': array([2, 3])}, array([12, 13]))
({'features': array([4, 5])}, array([14, 15]))


### tf.data TextLineDataset

In [15]:
def decode_csv(line):
    parsed_line = tf.decode_csv(line, [[0], [0]])
    label = parsed_line[-1:]  # Last element is the label
    del parsed_line[-1]       # Delete last element
    features = parsed_line
    d = dict(zip(["features"], features))
    return d, tf.squeeze(label) # estimator will run regardless squeeze or not
    #return d, label
dataset = tf.data.TextLineDataset('file.csv').map(decode_csv)
dataset = dataset.repeat(2).batch(2)
next_element = dataset.make_one_shot_iterator().get_next()

with tf.Session() as sess:
    for i in range(3):
        print(sess.run(next_element))

({'features': array([0, 1])}, array([10, 11]))
({'features': array([2, 3])}, array([12, 13]))
({'features': array([4, 5])}, array([14, 15]))
